# Lademanagementsimulation   


### Simulations Parameter

In [1]:
solarpeakleistung = 125 #Einheit Kilowattpeak (kWP)
anzahl_bevs_pro_tag = 10
ladeleistung_pro_bev = 11 #Einheit kW
maximale_ladezeit = 30 #Einheit Minuten
minute_interval = 15 #Einheit Minuten

### Simulations Eingangsdaten

In [ ]:
import simulationInputDataVisualisation
simulationInputDataVisualisation.create_jupyter_dash_app(solarpeakleistung)

### Verteilungsalgorithmus

In [ ]:
import simulationClasses
import simulationData
import simulationOutputDataVisualisation
import distributionAlgorithmForecastPolling
import postOptimization
import numpy as np

simulation_day = simulationClasses.SimulationDay(anzahl_bevs_pro_tag)
bev_data = simulationData.BevData()
table_dict = simulationData.TableDict()
simulation_data = simulationData.SimulationData()
    
day_in_minute_interval_steps = list(np.around(np.arange(480, 960 + 1, minute_interval), 1))
distributionAlgorithmForecastPolling.init_simulation(day_in_minute_interval_steps, minute_interval, simulation_data, 
                                                   simulation_day, solarpeakleistung)

postOptimization.start_post_optimization(minute_interval, simulation_day, solarpeakleistung, bev_data, table_dict, simulation_data,
                ladeleistung_pro_bev)

distributionAlgorithmForecastPolling.start_simulation(solarpeakleistung, ladeleistung_pro_bev,
                     simulation_day, bev_data, table_dict, simulation_data, minute_interval)

### Ergebnis Algorithmus

In [ ]:
simulationOutputDataVisualisation.print_algorithm_result(bev_data, simulation_data, anzahl_bevs_pro_tag, solarpeakleistung, minute_interval)

### Tabellarische Übersicht Park- und Ladeverlauf der BEVs

In [ ]:
simulationOutputDataVisualisation.create_tabular_overview_per_minute_slider(table_dict, minute_interval)

In [ ]:
import figureGeneration

figureGeneration.create_bev_number_figure(simulation_data)

In [ ]:
figureGeneration.create_charging_power_figure(simulation_day, solarpeakleistung, bev_data, minute_interval)